In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/sample_submission.csv
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train/00929.png
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train/00704.png
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train/00562.png
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train/00237.png
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train/00406.png
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train/00448.png
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train/00255.png
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train/00435.png
/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/tr

In [27]:
# import numpy as np
# import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

In [28]:
real_images_dir = '/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train'
fake_images_dir = '/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/fake_train'

# List the image files
real_images = os.listdir(real_images_dir)
fake_images = os.listdir(fake_images_dir)

# # Example: Load and display a real and a fake image using OpenCV
# real_img_path = os.path.join(real_images_dir, real_images[9])
# fake_img_path = os.path.join(fake_images_dir, fake_images[9])

# # Load the images using OpenCV
# real_img = cv2.imread(real_img_path)
# fake_img = cv2.imread(fake_img_path)

# # Convert BGR (default by OpenCV) to RGB for correct display in matplotlib
# real_img_rgb = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB)
# fake_img_rgb = cv2.cvtColor(fake_img, cv2.COLOR_BGR2RGB)

# # Display images using matplotlib
# plt.figure(figsize=(10,5))

# plt.subplot(1, 2, 1)
# plt.title("Real Image")
# plt.imshow(real_img_rgb)

# plt.subplot(1, 2, 2)
# plt.title("Fake Image")
# plt.imshow(fake_img_rgb)

# plt.show()

In [29]:
# # Convert an image to a NumPy array and normalize to [0, 1]
# real_img_array = np.array(real_img_rgb) / 255.0
# fake_img_array = np.array(fake_img_rgb) / 255.0

# print(real_img_array.shape)  # Example: (128, 128)
# # real_img_array[1,:]
# print(len(real_images))  # Example: (128, 128)

In [35]:
# real_img.shape

data_generation = ImageDataGenerator(rescale=1./255, validation_split=(0.15))
# data_generation = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest',
#     validation_split=(0.25)
# )



train_dataset = data_generation.flow_from_directory('/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/',target_size=(128, 128), batch_size=32, class_mode='binary',subset='training')
validate_dataset = data_generation.flow_from_directory('/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images',target_size=(128, 128), batch_size=32, class_mode='binary',subset='validation')

Found 1360 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


In [36]:
train_dataset.class_indices

{'fake_train': 0, 'real_train': 1}

In [37]:
model=tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  #
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D(2,2),                                  
#                                   tf.keras.layers.Dropout(0.3),
#                                   tf.keras.layers.BatchNormalization(),
                                  #
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  ##
                                  tf.keras.layers.Flatten(),
                                  ##
                                  tf.keras.layers.Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#                                   tf.keras.layers.Dropout(0.1),
                                  ##
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                 ])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │       401,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 457,793 (1.75 MB)

 Trainable params: 457,793 (1.75 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.001),metrics=['accuracy'])

In [44]:
model_fit=model.fit(train_dataset,steps_per_epoch=50,epochs=15,validation_data=validate_dataset)

Epoch 1/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.8484 - loss: 0.3912 - val_accuracy: 0.9167 - val_loss: 0.2247
Epoch 2/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.8740 - loss: 0.3641 - val_accuracy: 0.9208 - val_loss: 0.2558
Epoch 3/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.8885 - loss: 0.2947 - val_accuracy: 0.8625 - val_loss: 0.3930
Epoch 4/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.8881 - loss: 0.3084 - val_accuracy: 0.9500 - val_loss: 0.1850
Epoch 5/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.8812 - loss: 0.3332 - val_accuracy: 0.7583 - val_loss: 0.6258
Epoch 6/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.9021 - loss: 0.2752 - val_accuracy: 0.8708 - val_loss: 0.3507
Epoch 7/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.8944 - loss: 0.2902 - val_accuracy: 0.9417 - val_loss: 0.2162
Epoch 8/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.8942 - loss: 0.2587 - val_accuracy: 0.8875 - v

In [45]:
test_dir='/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/test_images/test_images'
# count=0
# for i in os.listdir(test_dir):
#     if (count>=100):
#         break
# #     img_path = os.path.join(test_dir, i)
#     img=image.load_img(test_dir+'//'+i,target_size=(128,128))
#     plt.imshow(img)
#     plt.show()
    
#     x = image.img_to_array(img)
#     x = np.expand_dims(x,axis=0)
#     images = np.vstack([x])
#     val = model.predict(images)
    
#     if(val>0.5):
#         print("1")
#     else:
#         print("0")
        
#     count+=1
    

In [46]:
test_gen = ImageDataGenerator(rescale=(1./255))
test_dataset = test_gen.flow_from_directory('/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/test_images/',target_size=(128, 128), batch_size=32, class_mode='binary')

Found 400 images belonging to 1 classes.


In [47]:
predictions = model.predict(test_dataset)
# predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = (predictions > 0.5).astype(int).flatten()
print(predicted_labels)
for i in range(400):
    if(predicted_labels[i]==1):
        predicted_labels[i]=0
    else:
        predicted_labels[i]=1
    i+=1
        
print(predicted_labels)


        

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
[1 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 0 1 1 1 0 0
 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1
 0 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0
 0 0 0 1 0 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0
 1 1 1 0 0 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 1
 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0
 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0 0 1 1
 1 1 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1
 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 1 0
 1 0 0 1 1 1 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1 1]
[0 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 1
 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 0 1 0 1 0 1 0 0 1 1 1 0 1 1 1 1 1 

In [48]:
# n = 400
# image_id = [0] * n
# for i in range(n):
# #     print(test_dataset[i])
#     image_id[i] = test_dataset[i][1].split('_')[1].split('.')[0]
    
# image_id = np.array(image_id)

image_id = [i for i in range(1, 401)]

# Convert each element to a string and sort them lexicographically
image_id = sorted(image_id, key=lambda x: str(x))

image_id = np.array(image_id)

print(image_id)


# Create a pandas dataframe
df = pd.DataFrame({
    'ID': image_id,       # List of image names or IDs
    'Target': predicted_labels   # Corresponding predictions
})

# Save to a CSV file
df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")

[  1  10 100 101 102 103 104 105 106 107 108 109  11 110 111 112 113 114
 115 116 117 118 119  12 120 121 122 123 124 125 126 127 128 129  13 130
 131 132 133 134 135 136 137 138 139  14 140 141 142 143 144 145 146 147
 148 149  15 150 151 152 153 154 155 156 157 158 159  16 160 161 162 163
 164 165 166 167 168 169  17 170 171 172 173 174 175 176 177 178 179  18
 180 181 182 183 184 185 186 187 188 189  19 190 191 192 193 194 195 196
 197 198 199   2  20 200 201 202 203 204 205 206 207 208 209  21 210 211
 212 213 214 215 216 217 218 219  22 220 221 222 223 224 225 226 227 228
 229  23 230 231 232 233 234 235 236 237 238 239  24 240 241 242 243 244
 245 246 247 248 249  25 250 251 252 253 254 255 256 257 258 259  26 260
 261 262 263 264 265 266 267 268 269  27 270 271 272 273 274 275 276 277
 278 279  28 280 281 282 283 284 285 286 287 288 289  29 290 291 292 293
 294 295 296 297 298 299   3  30 300 301 302 303 304 305 306 307 308 309
  31 310 311 312 313 314 315 316 317 318 319  32 32